# Base de datos de códigos postales del Servicio Postal Mexicano
## Introduccion
Tuve la fortuna de encontrar la base de datos en un repositorio de Github que alguien ya había minado previamente.
la URL al repositorio es https://github.com/redrbrt/sepomex-zip-codes
## Revision al dataset

In [1]:
# Librerias utilizadas
import pandas as pd
import sys
import os
import csv
import urllib

In [6]:
# Descarga de archivos a carpeta local
fuente = r'https://github.com/redrbrt/sepomex-zip-codes/raw/master/sepomex_abril-2016.csv'
destino = r'D:\PCCS\00_RawData\01_CSV\SEPOMEX\sepomex_abril-2016.csv'
urllib.request.urlretrieve(fuente, destino)


('D:\\PCCS\\00_RawData\\01_CSV\\SEPOMEX\\sepomex_abril-2016.csv',
 <http.client.HTTPMessage at 0x1c7141a860>)

In [15]:
# Importa el dataset a Python
dataset = pd.read_csv(destino)
dataset.head(3)

,idEstado,estado,idMunicipio,municipio,ciudad,zona,cp,asentamiento,tipo
0,1,Aguascalientes,1,Aguascalientes,Aguascalientes,Urbano,20000,Zona Centro,Colonia
1,1,Aguascalientes,1,Aguascalientes,Aguascalientes,Urbano,20010,Colinas del Rio,Fraccionamiento
2,1,Aguascalientes,1,Aguascalientes,Aguascalientes,Urbano,20010,Las Brisas,Fraccionamiento


In [16]:
# Corregir longitud de claves
dataset['idEstado'] = dataset.idEstado.apply(lambda x: str(int(x)).zfill(2))   # Correccion a 2 digitos (00)
dataset['idMunicipio'] = dataset.idMunicipio.apply(lambda x: str(int(x)).zfill(3))    # Correccion a 3 digitos (000)
dataset['cp'] = dataset.cp.apply(lambda x: str(int(x)).zfill(5))     # Correccion a 5 digitos (00000)
dataset['CVE_MUN'] = dataset[['idEstado', 'idMunicipio']].apply(lambda x: ''.join(x), axis=1)     # Crea CVE_MUN estandar
dataset = dataset.rename(columns={'cp': 'CP'})
dataset.head(3)

,idEstado,estado,idMunicipio,municipio,ciudad,zona,CP,asentamiento,tipo,CVE_MUN
0,01,Aguascalientes,001,Aguascalientes,Aguascalientes,Urbano,20000,Zona Centro,Colonia,01001
1,01,Aguascalientes,001,Aguascalientes,Aguascalientes,Urbano,20010,Colinas del Rio,Fraccionamiento,01001
2,01,Aguascalientes,001,Aguascalientes,Aguascalientes,Urbano,20010,Las Brisas,Fraccionamiento,01001


In [17]:
# Creo una copia de trabajo
SEPOMEX1 = dataset

Debido a que el estado y municipio pueden ser derivados desde la CVE_MUN de 5 dígitos, voy a eliminar las columnas:
* idEstado
* estado
* idMunicipio
* municipio

Tambien eliminaré otras columnas por diferentes motivos:

* Ciudad, pues esta será asignada acorde a la Clave SUN
* Zona, porque 'rural/urbano' no se utiliza para la construccion de parametros
* Asentamiento, porque es un nivel de desagregación que no se utiliza
* tipo, porque no se utiliza para la construccion de parametros

In [18]:
columnasnointeresantes = ['idEstado', 'estado', 'idMunicipio', 'municipio', 'ciudad', 'zona', 'asentamiento', 'tipo']
for columna in columnasnointeresantes:
    del(SEPOMEX1[columna])
orden = ['CP', 'CVE_MUN']
SEPOMEX1 = SEPOMEX1[orden]
SEPOMEX1.head(3)

,CP,CVE_MUN
0,20000,01001
1,20010,01001
2,20010,01001


De este dataset hay que eliminar renglones duplicados (como ejemplo, los renglones 1 y 2 impresos arriba), que existen porque la base de datos del SEPOMEX contiene un renglón individual cada colonia de las que integran un código postal. 

In [19]:
print('[{}] El dataset inició con {} filas, incluyendo repetidas'.format(len(SEPOMEX1), len(SEPOMEX1)))
SEPOMEX1 = SEPOMEX1.drop_duplicates(keep='first', subset=['CP'])
print('[{}] Al quitar las repetidas, el dataset queda con {} filas'.format(len(SEPOMEX1), len(SEPOMEX1)))

[145908] El dataset inició con 145908 filas, incluyendo repetidas
[32467] Al quitar las repetidas, el dataset queda con 32467 filas


In [20]:
SEPOMEX1.head()

,CP,CVE_MUN
0,20000,01001
1,20010,01001
6,20016,01001
10,20018,01001
11,20020,01001


In [22]:
# Guardar dataset como archivo excel
file = r'D:\PCCS\01_Dmine\Datasets\SEPOMEX\sepomex_CP_CVEMUN.xlsx'
writer = pd.ExcelWriter(file)
SEPOMEX1.to_excel(writer, sheet_name = 'DATOS')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------
